# Netflix Movies Recomendation System
BY Sheikh Md Abid
## Project Workflow

### 1. **Importing Libraries**
The necessary Python libraries such as `numpy`, `pandas`, and `ast` are imported to handle data processing and manipulation.

In [1]:
import numpy as np
import pandas as pd

### 2. Loading the Datasets
The datasets `tmdb_5000_movies.csv` and `tmdb_5000_credits.csv` are loaded using Pandas. These datasets contain information about movies, including titles, genres, cast, crew, and more, which will be used to build the recommendation system.

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### 3. Merging Datasets
The `movies` and `credits` datasets are merged on the movie `title` to combine relevant information from both datasets into a single dataframe. This allows for a more comprehensive analysis by linking movie metadata with corresponding cast and crew details.


In [5]:
movies = movies.merge(credits, on='title')
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### 4. Data Preprocessing
The merged dataset is reduced to the most relevant columns for building the recommendation system. These columns include `movie_id`, `title`, `overview`, `genres`, `keywords`, `cast`, and `crew`. This reduction helps to focus the analysis on the essential features needed for content-based recommendations.

In [6]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### 5. Handling Missing Data
To ensure clean and consistent data, rows with missing values in the selected columns are dropped. This step is crucial for maintaining the integrity of the dataset, as missing values could affect the accuracy of the recommendation system.


In [7]:
movies.dropna(inplace=True)

### 6. Data Transformation
The columns `genres`, `keywords`, `cast`, and `crew` contain nested JSON-like structures, which need to be converted into lists of strings for further processing. This transformation is done using the `ast.literal_eval()` function, along with custom helper functions, to extract the relevant information from these nested structures.

In [8]:
import ast

In [9]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [10]:
movies['genres'] = movies['genres'].apply(convert)

In [11]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### 7. Processing Cast and Crew
The `cast` and `crew` columns are refined to focus on the most relevant information:

- **Cast:** The `cast` column is limited to the top 3 actors, as they typically have the most significant impact on a movie's identity and appeal.
- **Crew:** The `crew` column is filtered to retain only the director's name, as the director plays a crucial role in shaping the movie's vision and style.

These transformations ensure that the tags used for recommendations are concise and focused on the key contributors to a movie.


In [12]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [13]:
movies['cast'] = movies['cast'].apply(convert)
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [14]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [15]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [16]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [17]:
movies.sample(3)

,movie_id,title,overview,genres,keywords,cast,crew
491,76649,Pompeii,"Set in 79 A.D., POMPEII tells the epic story o...","[Action, Adventure, History, Romance, Drama]","[gladiator, arena, gladiator fight, lava, roma...","[Kit Harington, Carrie-Anne Moss, Emily Browning]",[Paul W.S. Anderson]
4504,11598,The Kentucky Fried Movie,A series of loosely connected skits that spoof...,[Comedy],"[journalism, commercial, television, tv show, ...","[Evan C. Kim, Bong Soo Han, Marilyn Joi]",[John Landis]
2158,38,Eternal Sunshine of the Spotless Mind,"Joel Barish, heartbroken that his girlfriend u...","[Science Fiction, Drama, Romance]","[deja vu, regret, jealousy, amnesia, dream, op...","[Jim Carrey, Kate Winslet, Kirsten Dunst]",[Michel Gondry]


### 8. Removing Spaces in Tags
To ensure that the vectorization process works effectively, spaces in the tags (such as actor names, genres, keywords, etc.) are removed. This step prevents issues where multi-word tags might be treated as separate tokens, which could dilute their significance in the recommendation system.


In [18]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [19]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [20]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


### 9. Creating the Tags Column
A new column called `tags` is created by combining the content from the `overview`, `genres`, `keywords`, `cast`, and `crew` columns into a single string. This consolidated column serves as the primary input for generating movie recommendations, as it encapsulates all the key descriptive information about each movie.


In [21]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [22]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [23]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [24]:
movies['tags'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

In [25]:
new_dataframe= movies.drop(columns=['overview','genres','keywords','cast','crew'])
new_dataframe.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [26]:
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x: " ".join(x))
new_dataframe.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


### 10. Vectorization
The `tags` column is vectorized using `CountVectorizer` from Scikit-Learn. This process converts the text data in the `tags` column into a numerical format that can be used for similarity calculations. The `CountVectorizer` is configured to handle a maximum of 5000 features and remove English stop words, which helps in focusing on the most meaningful terms.


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [28]:
vector = cv.fit_transform(new_dataframe['tags']).toarray()

In [29]:
vector.shape

(4806, 5000)

### 11. Calculating Similarity
Cosine similarity is used to measure the similarity between movies based on their vectorized `tags`. This technique calculates how similar two vectors are by determining the cosine of the angle between them, providing a measure of how closely related two movies are in terms of their tags.


In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
similarity = cosine_similarity(vector)

In [32]:
similarity

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

### 12. Building the Recommendation Function
The `recommend()` function is designed to take a movie title as input and return a list of the top 5 similar movies based on content similarity. This function uses the similarity matrix to find movies that are most similar to the input movie.


In [33]:
def recommend(movie):
    index = new_dataframe[new_dataframe['title'].map(lambda x: x.lower()) == movie.lower()].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_dataframe.iloc[i[0]].title)

### 13. Example Usage
To get movie recommendations using the `recommend()` function, simply provide the title of a movie as input. For example, to find movies similar to "Avatar", use the following code:


In [35]:
recommend('Weekend')

The Thief and the Cobbler
Metropolis
Defendor
Hobo with a Shotgun
The Bubble


## Conclusion
This Netflix Movies Recommendation System effectively identifies similar movies based on content. By leveraging cosine similarity, the system ensures that the recommended movies closely match the input movie in terms of genres, keywords, cast, and crew. This approach provides users with personalized and relevant movie suggestions, enhancing their viewing experience.
